In [1]:
import pandas as pd

## Load Data

In [2]:
all_jobs_file = '../data/postings/all_jobs_v2.csv'
all_jobs = pd.read_csv(all_jobs_file)
all_jobs.head(5)

,Unnamed: 0,job_title,company,location,description,date_posted,type,categories,levels,expired,language,data_source,new_description,cleaned_description,expected_skills,cleaned_date_posted,minimum_education
0,0,Lead Software Engineer,Simon & Schuster,"New York, NY",Simon & Schuster is seeking a Lead Software En...,2019-09-11 18:59:59+00:00,Full Time,NaN,NaN,NaN,NaN,github,Simon and Schuster is seeking a Lead Software ...,Simon and Schuster is seeking a Lead Software ...,JAVASCRIPT PYTHON SQL LINUX AZURE AWS GITHUB N...,9/11/2019 18:59,master
1,1,Web Developer,Noe & Associates,"New York, NY",A leading international branding and design co...,2019-09-09 15:29:25+00:00,Full Time,NaN,NaN,NaN,NaN,github,A leading international branding and design co...,A leading international branding and design co...,JAVASCRIPT PHP HTML RUBY LINUX WORDPRESS HTML5...,9/9/2019 15:29,NaN
2,2,Front End Developer,Prosek Partners,"Fairfield, CT","Background\nProsek Partners, one of the top 25...",2018-11-19 19:40:01+00:00,Full Time,NaN,NaN,NaN,NaN,github,"Background Prosek Partners, one of the top 25 ...",Background Prosek Partners one of the top ...,JAVASCRIPT UI CSS UX WINDOWS HTML5 CSS3 DOM CR...,11/19/2018 19:40,NaN
3,3,Cybersecurity Firm Seeks Senior Developer,Aon Cyber Solutions,NY or LA,Aon Cyber Solutions is looking for a Senior De...,2018-11-19 15:44:44+00:00,Full Time,NaN,NaN,NaN,NaN,github,Aon Cyber Solutions is looking for a Senior De...,Aon Cyber Solutions is looking for a Senior De...,REACT NODEJS DJANGO GIT POSTGRESQL APACHE REST...,11/19/2018 15:44,bachelor
4,4,Lead Front End Developer,The Atlantic,"Washington, DC & New York, NY",The Atlantic is a digital-first media company ...,2018-11-14 22:16:02+00:00,Full Time,NaN,NaN,NaN,NaN,github,The Atlantic is a digital first media company ...,The Atlantic is a digital first media company ...,JAVASCRIPT HTML CSS NODEJS GIT CHARACTER ES6,11/14/2018 22:16,bachelor


## Standardize Dates

In [3]:
from dateutil.parser import parse
import datetime

date = all_jobs.iloc[1]['date_posted']

def standardize_dates(date):    
    dt = parse(date)
    date = datetime.date(dt.year, dt.month, dt.day)
    time = datetime.time(dt.hour, dt.minute, dt.second)
    return '{} {}'.format(
        date, time
    )

all_jobs['cleaned_date_posted'] = all_jobs.date_posted.apply(lambda date: standardize_dates(date))
all_jobs.head(3)

,job_title,company,location,description,date_posted,type,categories,levels,expired,language,data_source,new_description,cleaned_description,expected_skills,cleaned_date_posted
0,Lead Software Engineer,Simon & Schuster,"New York, NY",<p>Simon &amp; Schuster is seeking a Lead Soft...,2019-09-11 18:59:59+00:00,Full Time,NaN,NaN,NaN,NaN,github,Simon and Schuster is seeking a Lead Software ...,Simon and Schuster is seeking a Lead Software ...,JAVASCRIPT PYTHON SQL LINUX AZURE AWS GITHUB N...,2019-09-11 18:59:59
1,Web Developer,Noe & Associates,"New York, NY",<p>A leading international branding and design...,2019-09-09 15:29:25+00:00,Full Time,NaN,NaN,NaN,NaN,github,A leading international branding and design co...,A leading international branding and design co...,JAVASCRIPT PHP HTML RUBY LINUX WORDPRESS HTML5...,2019-09-09 15:29:25
2,Front End Developer,Prosek Partners,"Fairfield, CT","<p>Background</p>\n<p>Prosek Partners, one of ...",2018-11-19 19:40:01+00:00,Full Time,NaN,NaN,NaN,NaN,github,"Background Prosek Partners, one of the top 25 ...",Background Prosek Partners one of the top ...,JAVASCRIPT UI CSS UX WINDOWS HTML5 CSS3 DOM CR...,2018-11-19 19:40:01


## Standardize Locations

In [ ]:
from tqdm import tqdm
import requests

locations = []
for location in tqdm(all_jobs.location, total=len(all_jobs)):
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&sensor=false&key={}'.format(
        location,
        API_KEY
    )
    r = requests.get(url)
    result = r.json()['results']
    if len(result) > 0:
        result = result[0]
        if 'formatted_address' in result:
            result = result['formatted_address']
            result = ''.join([i for i in result if not i.isdigit()])
            result = ', '.join([x.strip() for x in result.split(',')])
            locations.append(result)
        else:
            locations.append(location)
    else:
        locations.append(location)

 73%|████████████████████████████████████████████████████████▊                     | 3516/4822 [15:26<04:52,  4.46it/s]

In [ ]:
all_jobs['location_cleaned'] = locations
all_jobs.head(3)

In [ ]:
all_jobs.to_csv('../data/postings/all_jobs_v3.csv', index=False)